<a href="https://colab.research.google.com/github/SophieShin/NLP_22_Fall/blob/main/%5BSSH%5Dlab09_seq2seq_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 9 – Seq2seq with Attention for Machine Translation

- We will build on the previous model on the **Multi30k** dataset between German and English by training a sequence-to-sequence model with *attention* that can translate German sentences into English.
- We'll be implementing the model from [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473). This model achieves a much better perplexity than the previous model.
- Uses bidirectional RNNs using GRU instead of LSTM.
- **Enable GPU for this lab.**
- Steps
  - Preprocessing using `spaCy` tokeniser
  - Build vocab using `build_vocab_from_iterator()`
  - Load and collate batches using `DataLoader()`
  - Pad shorter sentences with `pad_sequence()`
  - Seq2seq model with attention using a single layer GRU with a bidirectional RNN (2 RNNs).
  - Training & Evaluation (perplexity)

-  Main Source: [PyTorch](https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html) tutorial.

### Download required tokeniser and models

In [1]:
!python3 -m spacy download en_core_web_sm
!python3 -m spacy download de_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 4.7 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.6 MB 4.7 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


Install [*torchdata*](https://https://pytorch.org/data/beta/index.html) and restart runtime afterwards if required

In [2]:
!pip3 install torchdata # restart runtime afterwards if required

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.5 MB 4.7 MB/s 
     |████████████████████████████████| 140 kB 63.8 MB/s 
     |██████████████████████████████  | 834.1 MB 1.2 MB/s eta 0:00:47tcmalloc: large alloc 1147494400 bytes == 0x3a096000 @  0x7fc975771615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |████████████████████████████████| 890.2 MB 6.8 kB/s 
     |████████████████████████████████| 21.0 MB 1.3 MB/s 
     |████████████████████████████████| 849 kB 61.8 MB/s 
     |████████████████████████████████| 317.1 MB 32 kB/s 
     |████████████████████████████████| 557.1 MB 11 kB/s 
     |████████████████████████████████| 127 kB 70.8 MB/s 
  Attempting uninstall: url

Check *torchtext* version

In [3]:
!pip3 show torchtext

Name: torchtext
Version: 0.13.1
Summary: Text utilities and datasets for PyTorch
Home-page: https://github.com/pytorch/text
Author: PyTorch core devs and James Bradbury
Author-email: jekbradbury@gmail.com
License: BSD
Location: /usr/local/lib/python3.7/dist-packages
Requires: numpy, torch, tqdm, requests
Required-by: 


If the *torchtext* version is not 0.10.0, change the torchtext version to 0.10.0 and then, restart runtime if required.

In [4]:
# Takes ~2mins
!pip3 install torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 4.7 MB/s 
     |████████████████████████████████| 831.4 MB 2.6 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0
    Uninstalling torch-1.13.0:
      Successfully uninstalled torch-1.13.0
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
torchdata 0.5.0 requires torch==1.13.0, but you have torch 1.9.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.


In [7]:
import torchtext
print(torchtext.__version__) #make sure the version is 0.10.0 after restarting runtime

0.10.0


In [8]:
from torchtext.datasets import Multi30k

from torchtext.data.utils import get_tokenizer

import spacy


### Load the German and English spaCy model tokenisers


In [9]:
de_tokenizer = get_tokenizer('spacy', language="de_core_news_sm")
en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

### Load the Dataset

In [10]:
train_iter, valid_iter, test_iter = Multi30k(root='./data', split=('train', 'valid', 'test'), language_pair=('de', 'en'))

training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 801kB/s] 
validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 228kB/s]
mmt16_task1_test.tar.gz: 100%|██████████| 43.9k/43.9k [00:00<00:00, 218kB/s]


https://pytorch.org/text/stable/datasets.html#multi30k

train: 29000/
valid: 1014/
test: 1000

*root – Directory where the datasets are saved. Default: os.path.expanduser(‘~/.torchtext/cache’)

*split – split or splits to be returned. Can be a string or tuple of strings. Default: (‘train’, ‘valid’, ‘test’)

*language_pair – tuple or list containing src and tgt language. Available options are (‘de’,’en’) and (‘en’, ‘de’)

### View sample sentences

In [11]:
src_sentence, tgt_sentence = next(valid_iter)
print(src_sentence, tgt_sentence)


Eine Gruppe von Männern lädt Baumwolle auf einen Lastwagen
 A group of men are loading cotton onto a truck



In [12]:
src_sentence, tgt_sentence = next(train_iter)
print(src_sentence, tgt_sentence)


Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.
 Two young, White males are outside near many bushes.



### Specify the directories of the data

In [13]:
import os

root_dir = "./data/Multi30k"

train_path_de = os.path.join(root_dir, "train.de")
train_path_en = os.path.join(root_dir, "train.en")

valid_path_de = os.path.join(root_dir, "val.de")
valid_path_en = os.path.join(root_dir, "val.en")

test_path_de = os.path.join(root_dir, "test.de")
test_path_en = os.path.join(root_dir, "test.en")



In [14]:
# Print out the train, valid and test paths for German and English
print(f"train_path_de {train_path_de}")
print(f"train_path_en {train_path_en}")
print(f"valid_path_de {valid_path_de}")
print(f"valid_path_en {valid_path_en}")
print(f"test_path_de {test_path_de}")
print(f"test_path_en {test_path_en}")


train_path_de ./data/Multi30k/train.de
train_path_en ./data/Multi30k/train.en
valid_path_de ./data/Multi30k/val.de
valid_path_en ./data/Multi30k/val.en
test_path_de ./data/Multi30k/test.de
test_path_en ./data/Multi30k/test.en


### Create German and English Vocabs
- We are going to use `build_vocab_from_iterator()` which we have used in earlier labs.

In [15]:
import io

from torchtext.vocab import build_vocab_from_iterator

def yield_tokens(file_path):
  with io.open(file_path, encoding = 'utf-8') as f:
    for line in f:
      yield line.strip().split()

de_vocab = build_vocab_from_iterator(yield_tokens(train_path_de), specials=['<unk>', '<pad>', '<sos>', '<eos>'], min_freq=2)
de_vocab.set_default_index(de_vocab["<unk>"])

# Q1. Build the English vocab
# CODE Below

en_vocab = build_vocab_from_iterator(yield_tokens(train_path_en), specials=['<unk>', '<pad>', '<sos>', '<eos>'], min_freq=2)
en_vocab.set_default_index(de_vocab["<unk>"])

print("The length of the de_vocab is", len(de_vocab))

de_stoi = de_vocab.get_stoi()
print(f"The DE index of '<unk>' is {de_stoi['<unk>']}")
print(de_stoi)

de_itos = de_vocab.get_itos()
print(f"The DE token at index 2 is {de_itos[2]}")
print(de_itos)

print(f"The length of the en_vocab is {len(en_vocab)}")

en_stoi = en_vocab.get_stoi()
print(f"The EN index of '<unk>' is {en_stoi['<unk>']}")
print(en_stoi)

en_itos = en_vocab.get_itos()
print(f"The EN token at index 4 is {en_itos[4]}")
print(en_itos)


The length of the de_vocab is 9762
The DE index of '<unk>' is 0
{'„Thomas': 9760, '„The': 9759, '„Texas': 9758, '„Love“': 9756, '„Guitar': 9755, 'üppigen': 9754, 'übt,': 9751, 'überfliegt': 9745, 'örtlicher': 9741, 'Übungen.': 9736, 'Übungen': 9735, 'Überreste': 9734, 'Üben': 9732, 'zwirbelt': 9727, 'zweispurigen': 9725, 'zuschauen,': 9723, 'zusammenpassenden': 9722, 'zusammengedrängt': 9721, 'zusammengebaut.': 9719, 'zuhört': 9717, 'zugewandt': 9715, 'zufährt.': 9714, 'zotteliger': 9713, 'ziehen,': 9711, 'zerrissenen': 9710, 'zerlumpter': 9709, 'zerbrochenen': 9705, 'zerbrochene': 9704, 'zeichnen.': 9701, 'wütend': 9698, 'würden.': 9696, 'wälzt': 9693, 'wurden': 9692, 'wodurch': 9688, 'wirbeln': 9684, 'winkend': 9682, 'wichtiges': 9680, 'wetteifert': 9679, 'werdendem': 9677, 'welches': 9675, 'weißhaarige': 9674, 'weißene': 9672, 'weiße.': 9671, 'weiß-rotes': 9668, 'weiß-': 9666, 'weinroten': 9663, 'weht.': 9662, 'wartende': 9658, 'war.': 9655, 'vorüberfährt.': 9653, 'vornehmen': 9652,

In [ ]:
# Q2. Why are the lengths of de_vocab and en_vocab different?
# A2. An English word can be translated to multiple Deutsch words and vice versa.

# Q3. What do the methods get_stoi() and get_itos() do?
# A3. get_stoi(): Dictionary mapping tokens to indices.
# A3. get_itos(): List mapping indices to tokens.

### Process the Data Sets

In [16]:
import torch

def data_process(path_de, path_en):
  raw_de_iter = iter(io.open(path_de, encoding="utf8"))
  raw_en_iter = iter(io.open(path_en, encoding="utf8"))
  data = []
  for (raw_de, raw_en) in zip(raw_de_iter, raw_en_iter):
    de_tensor_ = torch.tensor([de_vocab[token] for token in de_tokenizer(raw_de)],
                            dtype=torch.long)
    en_tensor_ = torch.tensor([en_vocab[token] for token in en_tokenizer(raw_en)],
                            dtype=torch.long)
    data.append((de_tensor_, en_tensor_))
  return data

train_data = data_process(train_path_de, train_path_en)
val_data = data_process(valid_path_de, valid_path_en)
test_data = data_process(test_path_de, test_path_en)

In [20]:
train_data[:4]

[(tensor([  19,   80,  246,   29,  107,   20,  119,    6,   14,  104, 9631, 4202,
          1971,    0]),
  tensor([  17,   22,    0, 1315,  868,   14,   66,   77,  311, 4937,    0,    0])),
 (tensor([  77,   29,    8,  924, 2989,   13,    0, 1971,    0]),
  tensor([ 131,   36,    6,  321,  318,   14, 1344,    4,  741, 5525, 4637,    0,
             0])),
 (tensor([   4,   61,   25,  229,    6,   13, 8204,   67,  869, 1971,    0]),
  tensor([   5,   54,   31,  216,   64,    4,  221, 7299,    0,    0])),
 (tensor([   4,   10,    6,    5,   43,   40,   28,    9,   11,  724,    7,  825,
            13,  314, 1971,    0]),
  tensor([  5,  11,   6,   4,  28,  27,   9,  32,   8,   4, 705, 552,   4, 371,
            0,   0]))]

In [ ]:
# Q5. What is data_process() doing to the given two paths?
# A5. Each sentence(EN, DE) is tokenized then numericalize tokens into tensor.. Train/Val/Test data contain the pair of token indices. 

# Q6. Can you think of ONE other preprocessing that could be done on the this dataset?
# A6. I think we have done enough to tokenize a raw text sentence, build vocabulary, and numericalize tokens into tensor.

### DataLoader

Deals with loading data during training

https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html
The last torch specific feature we’ll use is the DataLoader, which is easy to use since it takes the data as its first argument. Specifically, as the docs say: DataLoader combines a dataset and a sampler, and provides an iterable over the given dataset. The DataLoader supports both map-style and iterable-style datasets with single- or multi-process loading, customizing loading order and optional automatic batching (collation) and memory pinning.

Please pay attention to collate_fn (optional) that merges a list of samples to form a mini-batch of Tensor(s). Used when using batched loading from a map-style dataset.

In [22]:
de_vocab['<sos>']

2

In [28]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128
PAD_IDX = de_vocab['<pad>']
SOS_IDX = de_vocab['<sos>']
EOS_IDX = de_vocab['<eos>']


def generate_batch(data_batch):
  de_batch, en_batch = [], []
  for (de_item, en_item) in data_batch:
    de_batch.append(torch.cat([torch.tensor([SOS_IDX]), de_item, torch.tensor([EOS_IDX])], dim=0))
    en_batch.append(torch.cat([torch.tensor([SOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim=0))
  de_batch = pad_sequence(de_batch, padding_value=PAD_IDX)
  en_batch = pad_sequence(en_batch, padding_value=PAD_IDX)
  return de_batch, en_batch

train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(val_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn=generate_batch)

https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_sequence.html

Pad a list of variable length Tensors with padding_value

pad_sequence stacks a list of Tensors along a new dimension, and pads them to equal length. For example, if the input is list of sequences with size L x * and if batch_first is False, and T x B x * otherwise.

B is batch size. It is equal to the number of elements in sequences. T is length of the longest sequence. L is length of the sequence. * is any number of trailing dimensions, including none.

In [ ]:
# Q7. Explain what generate_batch does to a batch of sentences
# A7. Adding SOS and EOS token in the beginning and at the end.
# Padding is also implied when the length of a sentence is shorter than the longest one. 


## NN Module: Encoder, Attention, Decoder, Seq2Seq
In the previous model, we used a multi-layered LSTM as the encoder and decoder. One downside of the previous model is that the decoder is trying to **cram lots of information into the hidden states**. Whilst decoding, the hidden state will need to contain information about the whole of the source sequence, as well as all of the tokens have been decoded (translated) so far. By alleviating some of this information compression, we can create a better model!

The model implemented here avoids this compression by allowing the decoder to **look at the entire source sentence (via its hidden states)** at each decoding step! How does it do this? It uses *attention*. 

Attention works by first, calculating an attention vector, $a$, which has the length of the source sentence. The attention vector has the property that each element is between 0 and 1, and the entire vector sums to 1. We then calculate a **weighted sum of our source sentence hidden states, $H$**, to get a weighted source vector, $w$. 

$$w = \sum_{i}a_ih_i$$

We calculate a new weighted source vector every time-step when decoding, using it as input to our decoder RNN as well as the linear layer to make a prediction. We'll explain how to do all of this during the tutorial.


### Encoder
In contrast to the previous model, we use a **single layer GRU and a bidirectional RNN**. GRU only requires and returns a hidden state, there is no cell state like in the LSTM.

$$\begin{align*}
h_t &= \text{GRU}(emb(x_t), h_{t-1})\\
(h_t, c_t) &= \text{LSTM}(emb(x_t), h_{t-1}, c_{t-1})\\
h_t &= \text{RNN}(emb(x_t), h_{t-1})
\end{align*}$$

From the equations above, it looks like the RNN and the GRU are identical. Inside the GRU, however, is a number of *gating mechanisms* that control the information flow in to and out of the hidden state (similar to an LSTM). Again, for more info, check out [this](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) excellent post. 

With a bidirectional RNN, we have two RNNs in each layer. A *forward RNN* going over the embedded sentence from left to right (shown below in green), and a *backward RNN* going over the embedded sentence from right to left (teal). All we need to do in code is set `bidirectional = True` and then pass the embedded sentence to the RNN as before. 

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq8.png?raw=1)

We now have:

$$\begin{align*}
h_t^\rightarrow &= \text{EncoderGRU}^\rightarrow(emb(x_t^\rightarrow),h_{t-1}^\rightarrow)\\
h_t^\leftarrow &= \text{EncoderGRU}^\leftarrow(emb(x_t^\leftarrow),h_{t-1}^\leftarrow)
\end{align*}$$

Where $x_0^\rightarrow = \text{<sos>}, x_1^\rightarrow = \text{guten}$ and $x_0^\leftarrow = \text{<eos>}, x_1^\leftarrow = \text{morgen}$.

As before, we only pass an input (`embedded`) to the RNN, which tells PyTorch to initialize both the forward and backward initial hidden states ($h_0^\rightarrow$ and $h_0^\leftarrow$, respectively) to a tensor of all zeros. We'll also get two context vectors, one from the forward RNN after it has seen the final word in the sentence, $z^\rightarrow=h_T^\rightarrow$, and one from the backward RNN after it has seen the first word in the sentence, $z^\leftarrow=h_T^\leftarrow$.

The RNN returns `outputs` and `hidden`. 

`outputs` is of size **[src len, batch size, hid dim * num directions]** where the first `hid_dim` elements in the third axis are the hidden states from the top layer forward RNN, and the last `hid_dim` elements are hidden states from the top layer backward RNN. We can think of the third axis as being the forward and backward hidden states concatenated together other, i.e. $h_1 = [h_1^\rightarrow; h_{T}^\leftarrow]$, $h_2 = [h_2^\rightarrow; h_{T-1}^\leftarrow]$ and we can denote all encoder hidden states (forward and backwards concatenated together) as $H=\{ h_1, h_2, ..., h_T\}$.

`hidden` is of size **[n layers * num directions, batch size, hid dim]**, where **[-2, :, :]** gives the top layer forward RNN hidden state after the final time-step (i.e. after it has seen the last word in the sentence) and **[-1, :, :]** gives the top layer backward RNN hidden state after the final time-step (i.e. after it has seen the first word in the sentence).

As the decoder is not bidirectional, it only needs a single context vector, $z$, to use as its initial hidden state, $s_0$, and we currently have two, a forward and a backward one ($z^\rightarrow=h_T^\rightarrow$ and $z^\leftarrow=h_T^\leftarrow$, respectively). We solve this by concatenating the two context vectors together, passing them through a linear layer, $g$, and applying the $\tanh$ activation function. 

$$z=\tanh(g(h_T^\rightarrow, h_T^\leftarrow)) = \tanh(g(z^\rightarrow, z^\leftarrow)) = s_0$$

**Note**: this is actually a deviation from the paper. Instead, they feed only the first backward RNN hidden state through a linear layer to get the context vector/decoder initial hidden state. This doesn't seem to make sense to me, so we have changed it.

As we want our model to look back over the whole of the source sentence we return `outputs`, the stacked forward and backward hidden states for every token in the source sentence. We also return `hidden`, which acts as our initial hidden state in the decoder.

In [23]:
import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import tensor


class Encoder(nn.Module):
    def __init__(self,
                 input_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: float):
        super().__init__()

        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout

        # Embedding layer 
        self.embedding = nn.Embedding(input_dim, emb_dim)

        # GRU
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)

        # Fully connected layer
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self,
                src: tensor) -> Tuple[tensor]:

        #src = [src len, batch size]

        embedded = self.dropout(self.embedding(src))

        #embedded = [src len, batch size, emb dim]

        # Forward pass through GRU
        outputs, hidden = self.rnn(embedded)

        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        # hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        # outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        # initial decoder hidden is final hidden state of the forwards and backwards 
        # encoder RNNs fed through a linear layer

        hidden_concat = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
        hidden = torch.tanh(self.fc(hidden_concat))

        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden




In [ ]:
# Q8. Why is the first input to the linear layer (self.fc), enc_hid_dim * 2?
# A8. Since we applied bidirections. 

### Attention

Using attention mechanism takes in the previous hidden state of the decoder, $s_{t-1}$, and ALL of the stacked forward and backward hidden states from the encoder, $H$. The layer will output an attention vector, $a_t$, that is the length of the source sentence, each element is between 0 and 1 and the entire vector sums to 1 (probability distribution).

This layer takes what we have decoded so far, $s_{t-1}$, and all of what we have encoded, $H$, to produce a vector, $a_t$, that represents which words in the source sentence we should pay the most attention to in order to correctly predict the next word to decode, $\hat{y}_{t+1}$.

Calculates the *attention score* between the previous decoder hidden state and the encoder hidden states. As our encoder hidden states are a sequence of $T$ tensors, and our previous decoder hidden state is a single tensor, the first thing we do is repeat the previous decoder hidden state $T$ times. We then calculate the attention/alignment score, $E_t$, between them by concatenating them together and passing them through a linear layer (attn) and a $\tanh$ activation function. This can be thought of as **how well each encoder hidden state "matches" the previous decoder hidden state.**

We can think of $v$ as the weights for a **weighted sum of the attention score across all encoder hidden states**. These weights tell us how much we should attend to each token in the source sequence. The parameters of $v$ are initialized randomly, but *learned* with the rest of the model via backpropagation. Note how $v$ is not dependent on time, and the same $v$ is used for each time-step of the decoding. Implement $v$ as a linear layer without a bias.

Finally, we ensure the attention vector fits the constraints of having all elements between 0 and 1 and the vector summing to 1 by passing it through a $\text{softmax}$ layer.

$$a_t = \text{softmax}(\hat{a_t})$$

This gives us the attention (distribution) over the source sentence!

Graphically, this looks something like below. This is for calculating the very first attention vector, where $s_{t-1} = s_0 = z$. The green/teal blocks represent the hidden states from both the forward and backward RNNs, and the attention computation is all done within the pink block.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq9.png?raw=1)


In [24]:
class Attention(nn.Module):
    def __init__(self,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 attn_dim: int):
        super().__init__()

        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        self.attn_in = (enc_hid_dim * 2) + dec_hid_dim

        self.attn = nn.Linear(self.attn_in, attn_dim)

    def forward(self,
                decoder_hidden: tensor,
                encoder_outputs: tensor) -> tensor:

        src_len = encoder_outputs.shape[0]

        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        a_score = torch.tanh(self.attn(torch.cat((
            repeated_decoder_hidden,
            encoder_outputs),
            dim = 2)))

        attention = torch.sum(a_score, dim=2)

        # This gives us the attention over the source sentence
        return F.softmax(attention, dim=1)


In [ ]:
# Q9. Try to figure out how this attention mechanism works. Looks at the last three lines of code.
# A9. In attention when the model is trying to predict the next word it searches for a set of positions in a source sentence 
# where the most relevant information is concentrated. The model then predicts next word based on context vectors 
# associated with these source positions and all the previous generated target words.

### Decoder
The decoder contains the attention layer, `attention`, which takes the previous hidden state, $s_{t-1}$, all of the encoder hidden states, $H$, and returns the attention vector, $a_t$.

We then use this attention vector to create a weighted source vector, $w_t$, denoted by `weighted`, which is a weighted sum of the encoder hidden states, $H$, using $a_t$ as the weights.

$$w_t = a_t H$$

The embedded input word, $d(y_t)$, the weighted source vector, $w_t$, and the previous decoder hidden state, $s_{t-1}$, are then all passed into the decoder RNN, with $d(y_t)$ and $w_t$ being concatenated together.

$$s_t = \text{DecoderGRU}(d(y_t), w_t, s_{t-1})$$

We then pass $d(y_t)$, $w_t$ and $s_t$ through the linear layer, $f$, to make a prediction of the next word in the target sentence, $\hat{y}_{t+1}$. This is done by concatenating them all together.

$$\hat{y}_{t+1} = f(d(y_t), w_t, s_t)$$

The image below shows decoding the first word in an example translation.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq10.png?raw=1)

The green/teal blocks show the forward/backward encoder RNNs which output $H$, the red block shows the context vector, $z = h_T = \tanh(g(h^\rightarrow_T,h^\leftarrow_T)) = \tanh(g(z^\rightarrow, z^\leftarrow)) = s_0$, the blue block shows the decoder RNN which outputs $s_t$, the purple block shows the linear layer, $f$, which outputs $\hat{y}_{t+1}$ and the orange block shows the calculation of the weighted sum over $H$ by $a_t$ and outputs $w_t$. Not shown is the calculation of $a_t$.

In [25]:
class Decoder(nn.Module):
    def __init__(self,
                 output_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: int,
                 attention: nn.Module):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)

        self.out = nn.Linear(self.attention.attn_in + emb_dim, output_dim)

        self.dropout = nn.Dropout(dropout)


    def _weighted_encoder_rep(self,
                              decoder_hidden: tensor,
                              encoder_outputs: tensor) -> tensor:

        a = self.attention(decoder_hidden, encoder_outputs)

        #a = [batch size, src len]

        a = a.unsqueeze(1)

        #a = [batch size, 1, src len]

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        #encoder_outputs = [batch size, src len, enc hid dim * 2]

        weighted_encoder_rep = torch.bmm(a, encoder_outputs)

        weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)

        return weighted_encoder_rep


    def forward(self,
                input: tensor,
                decoder_hidden: tensor,
                encoder_outputs: tensor) -> Tuple[tensor]:

        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]

        #input = [1, batch size]
        input = input.unsqueeze(0)


        #embedded = [1, batch size, emb dim]
        embedded = self.dropout(self.embedding(input))

        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden,
                                                          encoder_outputs)

        rnn_input = torch.cat((embedded, weighted_encoder_rep), dim = 2)

        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]

        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))

        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)

        output = self.out(torch.cat((output,
                                     weighted_encoder_rep,
                                     embedded), dim = 1))

        #output = [batch size, output dim]
        return output, decoder_hidden.squeeze(0)




### Seq2Seq
- Added something called **teacher forcing** - the ground truth label at current time step is used as input for the next time step
- Briefly going over all of the steps:
  - the outputs tensor is created to hold all predictions, $\hat{Y}$
  - the source sequence, $X$, is fed into the encoder to receive $z$ and $H$
  - the initial decoder hidden state is set to be the context vector, $s_0 = z = h_T$
  - we use a batch of <sos> tokens as the first input, $y_1$
  - we then decode within a loop:
    - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and all encoder outputs, $H$, into the decoder
    - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
    - we then decide if we are going to teacher force or not, setting the next input as appropriate

In [26]:
class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder: nn.Module,
                 decoder: nn.Module,
                 device: torch.device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self,
                src: tensor,
                trg: tensor,
                teacher_forcing_ratio: float = 0.5) -> tensor:

        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time

        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        #tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)

        # first input to the decoder is the <sos> token
        output = trg[0,:]

        for t in range(1, max_len):
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(output, hidden, encoder_outputs)

            #place predictions in a tensor holding predictions for each token
            outputs[t] = output

            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio

            #get the highest predicted token from our predictions
            top1 = output.max(1)[1]

            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            output = (trg[t] if teacher_force else top1)

        return outputs

### Create Model Instance and Optimiser
- This model follows the architecture described [here](https://arxiv.org/abs/1409.0473)
- You can find a significantly more commented version [here](https://github.com/SethHWeidman/pytorch-seq2seq/blob/master/3%20-%20Neural%20Machine%20Translation%20by%20Jointly%20Learning%20to%20Align%20and%20Translate.ipynb).
- Weights initialisation is introduced. We will initialize all biases to zero and all weights from $\mathcal{N}(0, 0.01)$.
- **Note:** this model is just an example model that can be used for language translation; we choose it because it is a standard model for the task, not because it is the recommended model to use for translation. 
- State-of-the-art models are currently based on Transformers; you can see PyTorch's capabilities for implementing Transformer layers [here](https://pytorch.org/docs/stable/nn.html#transformer-layers); and in particular, the "attention" used in the model below is different from the *multi-headed self-attention* present in a transformer model.

In [29]:
INPUT_DIM = len(de_vocab)
OUTPUT_DIM = len(en_vocab)

# > 12 mins | Test Loss: 3.063 | Test Perplexity:  21.394 |
# ENC_EMB_DIM = 256
# DEC_EMB_DIM = 256
# ENC_HID_DIM = 512
# DEC_HID_DIM = 512
# ATTN_DIM = 64
# ENC_DROPOUT = 0.5
# DEC_DROPOUT = 0.5

ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
ENC_HID_DIM = 256
DEC_HID_DIM = 256
ATTN_DIM = 32
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)

attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM)

dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)


# Initialise weights
def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01) # Initialise weights with values drawn from the normal distribution
        else:
            nn.init.constant_(param.data, 0) # Initialiase biases to 0


print(model.apply(init_weights))

optimizer = optim.Adam(model.parameters())

# Determine number of paramters
def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(9762, 128)
    (rnn): GRU(128, 256, bidirectional=True)
    (fc): Linear(in_features=512, out_features=256, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=768, out_features=32, bias=True)
    )
    (embedding): Embedding(7964, 128)
    (rnn): GRU(640, 256)
    (out): Linear(in_features=896, out_features=7964, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)
The model has 10,851,132 trainable parameters


### Loss Function
**Note:** When scoring the performance of a language translation model in particular, we have to tell the `nn.CrossEntropyLoss` function to ignore the indices where the target is simply padding.

In [30]:
PAD_IDX = en_stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

### Train and Evaluate Functions

In [31]:
import time


def train(model: nn.Module,
          iterator: torch.utils.data.DataLoader,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, (src, trg) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)

        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def evaluate(model: nn.Module,
             iterator: torch.utils.data.DataLoader,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for _, (src, trg) in enumerate(iterator):
            src, trg = src.to(device), trg.to(device)

            output = model(src, trg, 0) #turn off teacher forcing

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


### Let's Train!
- Added `perplexity` which is a common metrics for evaluating language models. We can add perplexity by exponentiating the loss.
- Training time: < 6mins on GPU


In [ ]:
import math

N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iter, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 's2s-model.pt')

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Perplexity: {math.exp(train_loss):7.3f} |')
    print(f'\t Val. Loss: {valid_loss:.3f} |   Val Perplexity: {math.exp(valid_loss):7.3f} |')


Epoch: 01 | Time: 0m 46s
	Train Loss: 4.853 | Train Perplexity: 128.062 |
	 Val. Loss: 4.433 |   Val Perplexity:  84.165 |
Epoch: 02 | Time: 0m 46s
	Train Loss: 4.032 | Train Perplexity:  56.391 |
	 Val. Loss: 4.277 |   Val Perplexity:  72.042 |
Epoch: 03 | Time: 0m 45s
	Train Loss: 3.705 | Train Perplexity:  40.648 |
	 Val. Loss: 4.031 |   Val Perplexity:  56.345 |
Epoch: 04 | Time: 0m 46s
	Train Loss: 3.425 | Train Perplexity:  30.715 |
	 Val. Loss: 3.781 |   Val Perplexity:  43.864 |
Epoch: 05 | Time: 0m 46s
	Train Loss: 3.113 | Train Perplexity:  22.481 |
	 Val. Loss: 3.507 |   Val Perplexity:  33.336 |
Epoch: 06 | Time: 0m 46s
	Train Loss: 2.838 | Train Perplexity:  17.076 |
	 Val. Loss: 3.351 |   Val Perplexity:  28.518 |
Epoch: 07 | Time: 0m 46s
	Train Loss: 2.615 | Train Perplexity:  13.662 |
	 Val. Loss: 3.247 |   Val Perplexity:  25.702 |
Epoch: 08 | Time: 0m 46s
	Train Loss: 2.448 | Train Perplexity:  11.569 |
	 Val. Loss: 3.146 |   Val Perplexity:  23.232 |
Epoch: 09 | Time

### Test the model on the test set using these "best" parameters

In [ ]:
model.load_state_dict(torch.load('s2s-model.pt'))

test_loss = evaluate(model, test_iter, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test Perplexity: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.115 | Test Perplexity:  22.523 |


See Ben Trevett's tutorial [here](https://github.com/bentrevett/pytorch-seq2seq/blob/master/3%20-%20Neural%20Machine%20Translation%20by%20Jointly%20Learning%20to%20Align%20and%20Translate.ipynb) for a better explanation of the steps taken for this model (it uses older code)